# Naughts and Crosses, the cooperative way

In [1]:
from naughtsandcrosses import *
from naughtandcrosses_cooperative import *
from mcts import MCTS

from IPython.display import clear_output
from time import sleep
from copy import deepcopy
import random

In [26]:
def play_one_game(state, p1, p2, show=True):
    s = deepcopy(state) # no side effects

    while not s.is_terminal():
        if show:
            print(s)

        action = p1.search(initial_state=s) if s.currentPlayer == 1 else p2.search(initial_state=s)
        s = s.take_action(action)

        if show:
            sleep(3) # so that you have time to see what's happening
            clear_output()

    reward = s.get_reward()
    if show:
        print(s)
        print(f"Game Over.\nReward: {reward}.")

    return s, int(reward)

## Sanity checks

It's probably a good idea to check if everything works as expected.

In [27]:
state = NaughtsAndCrossesState() # start with the 'X' player (i.e., 1)

strong_searcher = MCTS(iteration_limit=1000)
weak_searcher = MCTS(iteration_limit=1)

play_one_game(state, strong_searcher, weak_searcher); # ok, looks good, but we can do better


O | O |  
X | X | X
  |   |  

Game Over.
Reward: 1.0.


In [9]:
rewards = []
for n in range(100):
    state = NaughtsAndCrossesState() # start with the 'X' player (i.e., 1)

    strong_searcher = MCTS(iteration_limit=1000)
    weak_searcher = MCTS(iteration_limit=1)

    # switch the players, just for diversity's sake!
    _, r = play_one_game(state, weak_searcher, strong_searcher, show=False)
    rewards.append(r)

In [10]:
rewards.count(-1) / len(rewards) # very good! the strong searcher wins every time, as expected

1.0

It's a good idea to test the cooperative reward, while we're at it...

In [4]:
test_boards = [
    [[1, -1, 1], [-1, 1, -1], [1, -1, 1]],  # Perfect chequered pattern
    [[1, -1, 1], [-1, 1, 1], [1, -1, 1]],  # Incorrect pattern (two adjacent 1s)
    [[1, -1, 1], [-1, 0, -1], [1, -1, 1]],  # Not fully filled
    [[1, 1, 1], [-1, -1, -1], [1, 1, 1]],  # All rows same
    [[1, -1, 1], [1, -1, 1], [1, -1, 1]],  # All columns same
    [[-1, 1, -1], [1, -1, 1], [-1, 1, -1]], # Alternate starting symbol, chequered
]

In [5]:
instances = [NaughtsAndCrossesCoopState() for _ in range(len(test_boards))]

for b in range(len(test_boards)):
    instances[b].board = test_boards[b]

print(
    [(instances[i], instances[i].get_reward()) for i in range(len(test_boards))]
    )

[(
X | O | X
O | X | O
X | O | X
, 1), (
X | O | X
O | X | X
X | O | X
, 0), (
X | O | X
O |   | O
X | O | X
, 0), (
X | X | X
O | O | O
X | X | X
, 0), (
X | O | X
X | O | X
X | O | X
, 0), (
O | X | O
X | O | X
O | X | O
, 1)]


Looks ok to me.

Let's also check the terminating condition for the cooperative case.

In [6]:
test_boards = [
    [[1, -1, 1], [-1, 1, -1], [1, -1, 1]],  # Full board, should be terminal
    [[1, -1, 1], [-1, 0, -1], [1, -1, 1]],  # One empty cell, not terminal
    [[1, -1, -1], [-1, 1, 0], [1, -1, 1]],  # One empty cell, should be terminal as it cannot maintain the pattern
    [[0, 0, 0], [0, 0, 0], [0, 0, 0]],  # Empty board, not terminal
    [[1, 1, -1], [-1, -1, 1], [1, 1, -1]]  # Impossible to maintain pattern, should be terminal
]

In [21]:
instances = [NaughtsAndCrossesCoopState() for _ in range(len(test_boards))]

for b in range(len(test_boards)):
    instances[b].board = test_boards[b]

print(
    [(instances[i], instances[i].is_terminal()) for i in range(len(test_boards))]
    )

[(
X | O | X
O | X | O
X | O | X
, True), (
X | O | X
O |   | O
X | O | X
, False), (
X | O | O
O | X |  
X | O | X
, True), (
  |   |  
  |   |  
  |   |  
, False), (
X | X | O
O | O | X
X | X | O
, True)]


## Cooperation 

In theory, MCTS should work right away: the agents should both find out that cooperation is the only winning way, when expanding the tree!

Let's see if that works...

In [45]:
def starting_state(difficulty):
    state = NaughtsAndCrossesCoopState()  # starts with 'X' player

    if difficulty == 0:
        board_one_move_away = [
            [1, -1, 1],
            [-1, 0, -1],
            [1, -1, 1]  # needs naught
        ]
        state.board = board_one_move_away

    if difficulty == 1:
        board_two_moves_away = [
            [1, -1, 1],
            [-1, 0, -1], # needs cross
            [1, 0, 1]  # needs naught
        ]
        state.board = board_two_moves_away

    if difficulty == 2:
        board_three_moves_away = [
            [1, -1, 0],   # needs cross
            [-1, 0, -1],  # needs cross
            [1, 0, 1]    # needs naught
        ]
        state.board = board_three_moves_away

    if difficulty == 10:
        ipos = (random.randint(0, 2), random.randint(0, 2)) # choose position
        imov = random.choice([-1, 1])  # choose between 'O' and 'X'
        state.board[ipos[0]][ipos[1]] = imov

    return state

In [46]:
weak_searcher = MCTS(iteration_limit=1000)

play_one_game(starting_state(0), weak_searcher, weak_searcher);


X | O | X
O | X | O
X | O | X

Game Over.
Reward: 1.


It fails rather spectacularly and quickly, even when we only have three missing pieces! This isn't particularly surprising, though: the reward is as sparse as it can get (+1 at the end, 0 throughout) so, each time, it needs to go through a whole MCTS roll-out, till the end, to decide where to place the naught/cross!

What if we increase the iteration limit?

In [49]:
strong_searcher = MCTS(iteration_limit=1_000_000)

play_one_game(starting_state(1), strong_searcher, strong_searcher);


X | O | X
O | X | O
X | O | X

Game Over.
Reward: 1.


Something strange is happening: MCTS is taking much longer than necessary, considering how easy the problem is...